In [1]:

import os
from tqdm import tqdm
import cv2
import torch
import numpy as np

In [2]:
gt_path = '../testdata/labels/'
preds_path = '../testdata/preds/'

# get all imgs'name, remove the extension
images_folder1 = [f for f in os.listdir(gt_path) if
                  os.path.isfile(os.path.join(gt_path, f)) and f.lower().endswith(
                      ('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.gif'))]
images_names = [os.path.splitext(f)[0] for f in images_folder1]
tbar = tqdm(images_names)



  0%|          | 0/21 [00:00<?, ?it/s]

# Target Center Metric

In [3]:
from sosmetrics.metrics import TargetPrecisionRecallF1
# For test multiple format of input.
gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []

Metric = TargetPrecisionRecallF1(
    dis_thrs=[1, 10],
    conf_thr=0.5,
    match_alg='hungarian',
    second_match='plus_mask',
    # dilate_kernel=[7,7]
    
    debug = True
    )
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png")  # 假设所有图片都是PNG格式，或者你可以根据实际情况修改
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format

    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()

Reading image_name=0020: 100%|██████████| 21/21 [00:01<00:00, 11.36it/s]

bbox_iou=[[1.]]
mask_iou=[[0.86486486]]
eul_distance=[[0.2672813]]
________________________________________________________________________________
After second matche eul distance=[[0.40241643]]
________________________________________________________________________________
TargetPrecisionRecallF1.update() took 0.01s.
bbox_iou=[[1.]]
mask_iou=[[0.86486486]]
eul_distance=[[0.2672813]]
________________________________________________________________________________
After second matche eul distance=[[0.40241643]]
________________________________________________________________________________
TargetPrecisionRecallF1.update() took 0.00s.
bbox_iou=[[1. 0. 0.]
 [0. 1. 0.]]
mask_iou=[[0.96296296 0.         0.        ]
 [0.         1.         0.        ]]
eul_distance=[[ 0.08694401  9.409831   10.286567  ]
 [ 9.328961    0.          2.5       ]]
________________________________________________________________________________
After second matche eul distance=[[0.12398105        inf        inf

Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
TargetPrecisionRecallF1.update() took 0.00s each time.
+---------+----+----+----+------------------+---------------+----------------+
| Dis-Thr | TP | FP | FN | target_Precision | target_Recall | target_F1score |
+---------+----+----+----+------------------+---------------+----------------+
|   1.0   | 34 | 24 | 18 |     0.58621      |    0.65385    |    0.61818     |
|   2.0   | 42 | 16 | 10 |     0.72414      |    0.80769    |    0.76364     |
|   3.0   | 44 | 14 | 8  |     0.75862      |    0.84615    |    0.80000     |
|   4.0   | 44 | 14 | 8  |     0.75862      |    0.84615    |    0.80000     |
|   5.0   | 44 | 14 | 8  |     0.75862      |    0.84615    |    0.80000     |
|   6.0   | 44 | 14 | 8  |     0.75862      |    0.84615    |    0.80000     |
|   7.0   | 44 | 14 | 8  |     0.75862      |    0.84615    |    0.80000     |
|   8.0   | 44 | 14 | 8  |     0.75862

(array([0.5862069 , 0.72413793, 0.75862069, 0.75862069, 0.75862069,
        0.75862069, 0.75862069, 0.75862069, 0.75862069, 0.75862069]),
 array([0.65384615, 0.80769231, 0.84615385, 0.84615385, 0.84615385,
        0.84615385, 0.84615385, 0.84615385, 0.84615385, 0.84615385]),
 array([0.61818182, 0.76363636, 0.8       , 0.8       , 0.8       ,
        0.8       , 0.8       , 0.8       , 0.8       , 0.8       ]))

In [4]:
Metric.table

,dis_thrs,TP,FP,FN,Precision,Recall,F1,mean_Precision,mean_Recall,mean_F1
0,1.0,17.0,12.0,9.0,0.586207,0.653846,0.618182,0.737931,0.823077,0.778182
1,2.0,21.0,8.0,5.0,0.724138,0.807692,0.763636,0.737931,0.823077,0.778182
2,3.0,22.0,7.0,4.0,0.758621,0.846154,0.800000,0.737931,0.823077,0.778182
3,4.0,22.0,7.0,4.0,0.758621,0.846154,0.800000,0.737931,0.823077,0.778182
4,5.0,22.0,7.0,4.0,0.758621,0.846154,0.800000,0.737931,0.823077,0.778182
5,6.0,22.0,7.0,4.0,0.758621,0.846154,0.800000,0.737931,0.823077,0.778182
6,7.0,22.0,7.0,4.0,0.758621,0.846154,0.800000,0.737931,0.823077,0.778182
7,8.0,22.0,7.0,4.0,0.758621,0.846154,0.800000,0.737931,0.823077,0.778182
8,9.0,22.0,7.0,4.0,0.758621,0.846154,0.800000,0.737931,0.823077,0.778182
9,10.0,22.0,7.0,4.0,0.758621,0.846154,0.800000,0.737931,0.823077,0.778182


## Target Average Precision


In [7]:
from sosmetrics.metrics import TargetAveragePrecision
Metric = TargetAveragePrecision(
    dis_thrs=[1, 10],
    conf_thrs=10,
    match_alg='hungarian',
    second_match='plus_mask',
    # debug = True
    )
# For test multiple format of input.
gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png")  # 假设所有图片都是PNG格式，或者你可以根据实际情况修改
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format

    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()


Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
TargetAveragePrecision.update() took 0.02s each time.
+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| Dis—Thr |  1.0   |  2.0   |  3.0   |  4.0   |  5.0   |  6.0   |  7.0   |  8.0   |  9.0   |  10.0  |
+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|    AP   | 0.3030 | 0.4612 | 0.5467 | 0.5467 | 0.5689 | 0.5689 | 0.5689 | 0.5689 | 0.5689 | 0.5689 |
+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
TargetAveragePrecision(match_alg=hungarian, second_match=plus_mask, dilate_kernel=[0, 0])
Test image of list [hwc, hwc, ...]
TargetAveragePrecision.update() took 0.03s each time.
+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| Dis—Thr |  1.0   |  2.0   |  3.0   |

(array([[0.        , 0.6       , 0.6       , 0.5862069 , 0.5862069 ,
         0.56666667, 0.5       , 0.53333333, 0.55172414, 0.33333333],
        [0.        , 0.7       , 0.7       , 0.72413793, 0.72413793,
         0.7       , 0.7       , 0.7       , 0.68965517, 0.53333333],
        [0.        , 0.73333333, 0.73333333, 0.75862069, 0.75862069,
         0.73333333, 0.73333333, 0.73333333, 0.72413793, 0.63333333],
        [0.        , 0.73333333, 0.73333333, 0.75862069, 0.75862069,
         0.73333333, 0.73333333, 0.73333333, 0.72413793, 0.63333333],
        [0.        , 0.73333333, 0.73333333, 0.75862069, 0.75862069,
         0.73333333, 0.73333333, 0.73333333, 0.72413793, 0.66666667],
        [0.        , 0.73333333, 0.73333333, 0.75862069, 0.75862069,
         0.73333333, 0.73333333, 0.73333333, 0.72413793, 0.66666667],
        [0.        , 0.73333333, 0.73333333, 0.75862069, 0.75862069,
         0.73333333, 0.73333333, 0.73333333, 0.72413793, 0.66666667],
        [0.        , 0.7333

In [8]:
Metric.table

,0,1,2,3,4,5,6,7,8,9
Dis-Thr,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000,8.000000,9.000000,10.000000
AP,0.302962,0.461229,0.546729,0.546729,0.568877,0.568877,0.568877,0.568877,0.568877,0.568877


# PD_FA Metric

In [9]:
# For test multiple format of input.
from sosmetrics.metrics import TargetPdPixelFa


gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []

Metric = TargetPdPixelFa(
    dis_thrs=[1, 10],
    conf_thr=0.5,
    # match_alg='forloop',
    match_alg='hungarian',
    second_match='plus_mask'
    # debug = True
    )
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png") 
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format
    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()


Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
TargetPdPixelFa.update() took 0.00s each time.
+-----------+----+----+-----+---------+-----------+-------------+
| Threshold | TD | AT |  FD |    NP   | target_Pd |   pixel_Fa  |
+-----------+----+----+-----+---------+-----------+-------------+
|    1.0    | 34 | 52 | 392 | 2752512 |  0.65385  | 1.42415e-04 |
|    2.0    | 42 | 52 | 174 | 2752512 |  0.80769  | 6.32150e-05 |
|    3.0    | 44 | 52 | 108 | 2752512 |  0.84615  | 3.92369e-05 |
|    4.0    | 44 | 52 | 108 | 2752512 |  0.84615  | 3.92369e-05 |
|    5.0    | 44 | 52 | 108 | 2752512 |  0.84615  | 3.92369e-05 |
|    6.0    | 44 | 52 | 108 | 2752512 |  0.84615  | 3.92369e-05 |
|    7.0    | 44 | 52 | 108 | 2752512 |  0.84615  | 3.92369e-05 |
|    8.0    | 44 | 52 | 108 | 2752512 |  0.84615  | 3.92369e-05 |
|    9.0    | 44 | 52 | 108 | 2752512 |  0.84615  | 3.92369e-05 |
|    10.0   | 44 | 52 | 108 | 2752512 |  0.8

(array([0.65384615, 0.80769231, 0.84615385, 0.84615385, 0.84615385,
        0.84615385, 0.84615385, 0.84615385, 0.84615385, 0.84615385]),
 array([1.42415365e-04, 6.32149833e-05, 3.92368862e-05, 3.92368862e-05,
        3.92368862e-05, 3.92368862e-05, 3.92368862e-05, 3.92368862e-05,
        3.92368862e-05, 3.92368862e-05]))

In [10]:
Metric.table

,dis_thrs,TD,AT,FD,NP,target_Pd,pixel_Fa
0,1.0,17.0,26.0,196.0,1376256.0,0.653846,0.000142
1,2.0,21.0,26.0,87.0,1376256.0,0.807692,0.000063
2,3.0,22.0,26.0,54.0,1376256.0,0.846154,0.000039
3,4.0,22.0,26.0,54.0,1376256.0,0.846154,0.000039
4,5.0,22.0,26.0,54.0,1376256.0,0.846154,0.000039
5,6.0,22.0,26.0,54.0,1376256.0,0.846154,0.000039
6,7.0,22.0,26.0,54.0,1376256.0,0.846154,0.000039
7,8.0,22.0,26.0,54.0,1376256.0,0.846154,0.000039
8,9.0,22.0,26.0,54.0,1376256.0,0.846154,0.000039
9,10.0,22.0,26.0,54.0,1376256.0,0.846154,0.000039


# PixelNormalizeIoU

In [11]:
from sosmetrics.metrics import PixelNormalizedIoU
Metric = PixelNormalizedIoU(
    conf_thr=0.5,
    # debug = True
    )
# For test multiple format of input.
gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png")  # 假设所有图片都是PNG格式，或者你可以根据实际情况修改
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format

    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()


Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
PixelNormalizedIoU.update() took 0.00s each time.
+----------+
| nIoU-0.5 |
+----------+
|  0.7178  |
+----------+
PixelNormalizedIoU(conf_thr=0.5)
Test image of list [hwc, hwc, ...]
PixelNormalizedIoU.update() took 0.00s each time.
+----------+
| nIoU-0.5 |
+----------+
|  0.7178  |
+----------+
PixelNormalizedIoU(conf_thr=0.5)
Test image_path of list, [img_path, img_path, ...]
PixelNormalizedIoU.update() took 0.00s each time.
+----------+
| nIoU-0.5 |
+----------+
|  0.7178  |
+----------+
PixelNormalizedIoU(conf_thr=0.5)
Test image of np.array, bhwc
PixelNormalizedIoU.update() took 0.00s each time.
+----------+
| nIoU-0.5 |
+----------+
|  0.7178  |
+----------+
PixelNormalizedIoU(conf_thr=0.5)
Test image of tensor, bchw
PixelNormalizedIoU.update() took 0.00s each time.
+----------+
| nIoU-0.5 |
+----------+
|  0.7178  |
+----------+
PixelNormalizedIoU(conf_thr=0.5)
T

0.7178133052159646

In [12]:
Metric.table

,nIoU
0,0.717813


## HybridNormalizedIoU

In [13]:
from sosmetrics.metrics import HybridNormalizedIoU
Metric = HybridNormalizedIoU(
    conf_thr=0.5,
    dis_thrs = [1,10],
    match_alg='hungarian',
    second_match='plus_mask'
    # debug = True
    )
# For test multiple format of input.
gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png")  # 假设所有图片都是PNG格式，或者你可以根据实际情况修改
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format

    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()


Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
HybridNormalizedIoU.update() took 0.00s each time.
+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| Dis—Thr  |  1.0   |  2.0   |  3.0   |  4.0   |  5.0   |  6.0   |  7.0   |  8.0   |  9.0   |  10.0  |
+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| nIoU-0.5 | 0.5802 | 0.6610 | 0.6785 | 0.6785 | 0.6785 | 0.6785 | 0.6785 | 0.6785 | 0.6785 | 0.6785 |
+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
HybridNormalizedIoU(conf_thr=0.5, match_alg=hungarian, second_match=plus_mask, dilate_kernel=[0, 0])
Test image of list [hwc, hwc, ...]
HybridNormalizedIoU.update() took 0.00s each time.
+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| Dis—Thr  |  1.0   |  2.0 

KeyboardInterrupt: 

In [ ]:
Metric.table

# Pixel ROC Precision Recall Metric

In [ ]:
from sosmetrics.metrics import PixelROCPrecisionRecall
Metric = PixelROCPrecisionRecall(
    conf_thrs=30,
    # debug = True
    )
# For test multiple format of input.
gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png")  # 假设所有图片都是PNG格式，或者你可以根据实际情况修改
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format

    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()


In [ ]:
Metric.table

# Piexl Precision Recall F1 IoU 

In [ ]:
from sosmetrics.metrics import PixelPrecisionRecallF1IoU
Metric = PixelPrecisionRecallF1IoU(
    conf_thr=0.5,
    # debug = True
    )
# For test multiple format of input.
gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png")  # 假设所有图片都是PNG格式，或者你可以根据实际情况修改
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format

    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()


In [ ]:
Metric.table